<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/single_eye_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# read data from directory

In [1]:
#connect drive with colab notebook
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#show content in working directory
ls

drive/  sample_data/


#extract data

In [0]:
import pandas as pd

In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:
#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O) 

In [65]:
#print first 5 rows of annotation file
annotation.head(15)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
5,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
6,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,1,0,0,0,0,0,1
7,7,60,Female,7_left.jpg,7_right.jpg,drusen,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
8,8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
9,9,54,Male,9_left.jpg,9_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1


In [8]:
annotation['Left-Diagnostic Keywords'].value_counts().head(30)

normal fundus                                                      1433
moderate non proliferative retinopathy                              366
mild nonproliferative retinopathy                                   220
cataract                                                            135
pathological myopia                                                 111
lens dust，normal fundus                                             110
glaucoma                                                            104
dry age-related macular degeneration                                 93
severe nonproliferative retinopathy                                  70
drusen                                                               66
epiretinal membrane                                                  54
hypertensive retinopathy                                             53
macular epiretinal membrane                                          49
normal fundus，lens dust                                         

In [0]:


left_image=[]
left_label=[]
for i,j in zip(annotation['Left-Fundus'],annotation['Left-Diagnostic Keywords']):
  #print(i,j)
  if j=='normal fundus':left_label.append('N'),left_image.append(i)
  elif any(j==s for s in ['dry age-related macular degeneration' , 'wet age-related macular degeneration']): left_label.append('A'),left_image.append(i)
  elif any(j==s for s in ['diabetic retinopathy' , 'moderate non proliferative retinopathy' ,'mild nonproliferative retinopathy']):left_label.append('D'),left_image.append(i)
  elif any(j==s for s in ['glaucoma', 'suspected glaucoma']):left_label.append('G'),left_image.append(i)
  elif j=='cataract':left_label.append('C'),left_image.append(i)
  elif j=='hypertensive retinopathy':left_label.append('H'),left_image.append(i) 
  elif j=='pathological myopia':left_label.append('M'),left_image.append(i)
  else: left_label.append('O'),left_image.append(i)
  
  

In [51]:
print(left_image[:15])
print(left_label[:15])
print(set(left_label))

['0_left.jpg', '1_left.jpg', '2_left.jpg', '3_left.jpg', '4_left.jpg', '5_left.jpg', '6_left.jpg', '7_left.jpg', '8_left.jpg', '9_left.jpg', '10_left.jpg', '11_left.jpg', '12_left.jpg', '13_left.jpg', '14_left.jpg']
['C', 'N', 'O', 'N', 'O', 'D', 'O', 'O', 'N', 'N', 'O', 'O', 'O', 'M', 'N']
{'G', 'D', 'O', 'M', 'H', 'A', 'C', 'N'}


In [52]:
print(len(left_image))
print(len(left_label))

3500
3500


In [0]:
left_df=pd.DataFrame(zip(left_image,left_label),columns=['Image','Label'])

In [75]:
left_df.head(5)

,Image,Label
0,0_left.jpg,C
1,1_left.jpg,N
2,2_left.jpg,O
3,3_left.jpg,N
4,4_left.jpg,O


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
enc = MultiLabelBinarizer(classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'])
encoded_labels=enc.fit_transform(left_df['Label'])


In [0]:
df=pd.DataFrame(data=encoded_labels[0:,0:],index=[i for i in range(encoded_labels.shape[0])],columns=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'])

In [70]:
df.head()

,N,D,G,C,A,H,M,O
0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1


In [0]:
left_df=pd.concat([left_df,df],axis=1)


In [81]:
left_df.head()

,Image,Label,N,D,G,C,A,H,M,O
0,0_left.jpg,C,0,0,0,1,0,0,0,0
1,1_left.jpg,N,1,0,0,0,0,0,0,0
2,2_left.jpg,O,0,0,0,0,0,0,0,1
3,3_left.jpg,N,1,0,0,0,0,0,0,0
4,4_left.jpg,O,0,0,0,0,0,0,0,1


In [0]:


right_image=[]
right_label=[]
for i,j in zip(annotation['Right-Fundus'],annotation['Right-Diagnostic Keywords']):
  #print(i,j)
  if j=='normal fundus':right_label.append('N'),right_image.append(i)
  elif any(j==s for s in ['dry age-related macular degeneration' , 'wet age-related macular degeneration']): right_label.append('A'),right_image.append(i)
  elif any(j==s for s in ['diabetic retinopathy' , 'moderate non proliferative retinopathy' ,'mild nonproliferative retinopathy']):right_label.append('D'),right_image.append(i)
  elif any(j==s for s in ['glaucoma', 'suspected glaucoma']):right_label.append('G'),right_image.append(i)
  elif j=='cataract':right_label.append('C'),right_image.append(i)
  elif j=='hypertensive retinopathy':right_label.append('H'),right_image.append(i) 
  elif j=='pathological myopia':right_label.append('M'),right_image.append(i)
  else: right_label.append('O'),right_image.append(i)
  
  

In [84]:
print(set(right_label))

{'G', 'D', 'O', 'M', 'A', 'C', 'H', 'N'}


In [85]:
right_df=pd.DataFrame(zip(right_image,right_label),columns=['Image','Label'])
right_df.head()

,Image,Label
0,0_right.jpg,N
1,1_right.jpg,N
2,2_right.jpg,D
3,3_right.jpg,O
4,4_right.jpg,D


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
enc = MultiLabelBinarizer(classes=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'])
encoded_labels=enc.fit_transform(right_df['Label'])


In [0]:
df=pd.DataFrame(data=encoded_labels[0:,0:],index=[i for i in range(encoded_labels.shape[0])],columns=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'])

In [88]:
right_df=pd.concat([right_df,df],axis=1)
right_df.head()

,Image,Label,N,D,G,C,A,H,M,O
0,0_right.jpg,N,1,0,0,0,0,0,0,0
1,1_right.jpg,N,1,0,0,0,0,0,0,0
2,2_right.jpg,D,0,1,0,0,0,0,0,0
3,3_right.jpg,O,0,0,0,0,0,0,0,1
4,4_right.jpg,D,0,1,0,0,0,0,0,0


In [0]:
right_df.to_csv('right_annotation.csv')
left_df.to_csv('left_annotation.csv')